# Package installations

In [ ]:
!apt-get update -y
!apt-get install ffmpeg -y

In [ ]:
!pip install -r requirements.txt 

In [ ]:
#!pip install -U --force-reinstall --no-cache git+https://github.com/neel04/NeMo.git@dev#egg=nemo_toolkit[tts]
!pip install git+https://github.com/neel04/NeMo.git@dev#egg=nemo_toolkit[tts]

# Setup

These are some debug flags. You can turn them on for performance benchmarking:

```py
%env TORCH_WARN=1
%env PT_XLA_DEBUG_LEVEL=1
%env PJRT_DEVICE=TPU
%env TORCHDYNAMO_VERBOSE=0
```

In [1]:
import torch.nn.functional as F
import soundfile as sf

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel

[NeMo W 2025-01-07 19:24:06 nemo_logging:361] /usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
      from .autonotebook import tqdm as notebook_tqdm
    


In [2]:
import logging
logging.basicConfig(level = logging.INFO)
logging.getLogger("nemo_logger").setLevel(logging.ERROR)

In [50]:
import time
import torch
import torch_xla
import torchvision
import torch_xla.debug.metrics as met
import torch_xla.core.xla_model as xm

from contextlib import ContextDecorator
from dataclasses import dataclass
from typing import Optional

device = torch_xla.device()

torch_xla.experimental.eager_mode(False)

# Model definition

In [4]:
!rm -rf ./compilation_cache/
import torch_xla.runtime as xr
xr.initialize_cache('./compilation_cache', readonly=False)

In [5]:
spec_generator = FastPitchModel.from_pretrained("nvidia/tts_en_fastpitch").to(device)
model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan").to(device)

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
INFO:NeMo-text-processing:Creating ClassifyFst grammars.


Next, we convert the model to bf16. This utility function can also be configured to ignore certain weights which are better in full precision 

In [6]:
def convert_to_dtype(model, dtype=torch.float16):
    # Convert model parameters to FP16
    model = model.to(dtype)
    
    # Keep norm layers in FP32 (for future models)
    for name, param in model.named_parameters():
        if 'bn' in name:
            param.data = param.data.float()
            param.grad = param.grad.float()
    
    return model

model = convert_to_dtype(model)
spec_generator = convert_to_dtype(spec_generator)

Setting the model to `eval` (inferencing) mode:

In [7]:
spec_generator = spec_generator.to(device)
model = model.to(device)

spec_generator.eval()
model.eval()

HifiGanModel(
  (audio_to_melspec_precessor): FilterbankFeatures()
  (trg_melspec_fn): FilterbankFeatures()
  (generator): Generator(
    (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
    (ups): ModuleList(
      (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
      (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
      (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
      (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    )
    (resblocks): ModuleList(
      (0): ModuleList(
        (0): ResBlock1(
          (convs1): ModuleList(
            (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
            (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
          )
          (convs2): ModuleList(
            (0

## Helper functions

Here, we expose some functions for debugging & optimization reasons. One may need to partially compile some parts of the model depending on the architecture, usecase and performance. Thus we expose some utilities that we modify for better performance on TPU hardware.

In [8]:
@torch.compile(backend="openxla", fullgraph=True)
def log_to_duration(log_dur, min_dur, max_dur, mask):
    dur = torch.clamp(torch.exp(log_dur) - 1.0, min_dur, max_dur)
    dur *= mask.squeeze(2)
    return dur
    
@torch.compile(backend='openxla', fullgraph=True)
def compiled_device_transfer(x, device):
    return x.to(device)

This is simply a context manager for easy benchmarking of torch XLA code

In [51]:
@dataclass
class BenchmarkResult:
    time_ms: float
    time_sec: float
    output: Optional[any] = None

class benchmark_xla(ContextDecorator):
    def __init__(self, name: str = "Benchmark", silent: bool = False):
        self.name = name
        self.silent = silent
        self.result: Optional[BenchmarkResult] = None
        self.output = None
        
    def __enter__(self):
        self.start_time = time.time()
        return self
        
    def __exit__(self, *exc):
        xm.mark_step()
        xm.wait_device_ops()
            
        time_taken = time.time() - self.start_time
        
        self.result = BenchmarkResult(
            time_ms=round(time_taken * 1000, 3),
            time_sec=round(time_taken, 3),
            output=self.output
        )
        
        if not self.silent:
            print(f"{self.name} - Time taken: {self.result.time_ms} ms ({self.result.time_sec} seconds)")
            
        return False

## Forward pass

This function isn't `JIT`-compiled, however it does consume a static max length. Make sure inputs aren't clipped due to the static length. A max-length of `1024` should be good enough for <10s of audio - or even more.

In [9]:
Fastpitch = spec_generator.fastpitch # convenience

STATIC_MAX_LENGTH: int = 768

In [10]:
def static_regulate_len(
    durations,
    enc_out,
    pace: float = 1.0,
    mel_max_len: int = None,
    group_size: int = 1,
    dur_lens: torch.tensor = None,
    max_allowed_len: int = STATIC_MAX_LENGTH,
):
    """XLA-optimized version of regulate_len that minimizes dynamic operations.
    Uses a pre-defined maximum length to avoid dynamic arange operations."""
    dtype = enc_out.dtype
    device = enc_out.device
    
    # Static division and floor operations
    reps = (durations.float() / pace + 0.5).floor()#TODO: investigate .long()
    dec_lens = reps.sum(dim=1)
    
    # Instead of computing max_len dynamically, use max_allowed_len
    # This ensures static shape for XLA
    
    # Pre-compute pad and cumsum in a more static way
    padded_reps = torch.nn.functional.pad(reps, (1, 0, 0, 0), value=0.0)
    reps_cumsum = torch.cumsum(padded_reps, dim=1)[:, None, :]
    reps_cumsum = reps_cumsum.to(dtype=dtype, device=device)
    
    # Use static pre-computed range tensor instead of dynamic arange
    # This could be moved outside the function if needed
    static_range = torch.arange(max_allowed_len, device=device)[None, :, None]
    
    # Compute multiplication mask with fewer dynamic operations
    mult = (reps_cumsum[:, :, :-1] <= static_range) & (reps_cumsum[:, :, 1:] > static_range)
    mult = mult.to(dtype)
    
    # Final matrix multiplication
    enc_rep = torch.matmul(mult, enc_out)
    
    # Trim to actual length needed
    length_mask = torch.arange(enc_rep.shape[1], device=enc_rep.device)[None, :] < dec_lens[:, None]
    enc_rep = enc_rep * length_mask.unsqueeze(-1)
    
    return enc_rep, dec_lens, dec_lens.max()

`fp_prediction` is the fused inference function which consumes a `parsed: Tensor` textual input and returns the raw (padded) audio waveform. 

We also propogate `real_len: int` output which can be used to reconstruct how much we need to slice off the raw audio waveform to remove padding.

In [11]:
@torch.compile(backend='openxla', fullgraph=True)
def fp_prediction(parsed: torch.Tensor, conditioning=None):
    with torch.no_grad():
        parsed = parsed.to(device)     
        enc_out, enc_mask = Fastpitch.encoder(input=parsed, conditioning=conditioning)
        enc_out, enc_mask = enc_out.to(torch.float16), enc_mask.to(torch.float16)
        
        # Duration prediction
        log_durs_predicted = Fastpitch.duration_predictor(enc_out, enc_mask, conditioning=conditioning)
        durs_predicted = log_to_duration(
            log_dur=log_durs_predicted, 
            min_dur=Fastpitch.min_token_duration, 
            max_dur=Fastpitch.max_token_duration, 
            mask=enc_mask
        )
        
        # Pitch prediction
        pitch_predicted = Fastpitch.pitch_predictor(enc_out, enc_mask, conditioning=conditioning)
        pitch_emb = Fastpitch.pitch_emb(pitch_predicted.unsqueeze(1))
        
        # Combine encoder output and pitch embedding
        enc_out = enc_out + pitch_emb.transpose(1, 2)
        
        len_regulated, dec_lens, real_len = static_regulate_len(durs_predicted, enc_out, pace=1.0)
        dec_out, _ = Fastpitch.decoder(input=len_regulated, seq_lens=dec_lens, conditioning=conditioning)
        dec_out = dec_out.to(torch.float16)
        
        spect = Fastpitch.proj(dec_out).transpose(1, 2) # Obtain spectrogram

        audio = model.convert_spectrogram_to_audio(spec=spect)

        #audio = audio.to('cpu')
        audio = compiled_device_transfer(audio, torch.device('cpu'))

        return audio, real_len

In [57]:
def parse_text(text: str, padlen: int = 128):   
    _device = torch.device('cpu')
    torch.set_default_device('cpu')

    parsed = spec_generator.parse(text).cpu()
    
    if parsed.shape[1] > padlen:
        print(f'WARNING: Input padding is insufficient for text size. Recommend doublind paddling length.')
        
    parsed = F.pad(parsed.cpu(), (0, padlen - parsed.shape[1]), value=0).long()

    return parsed.cpu()

# Inference

`infer_e2e` is the exposed function that directly consumes `text: str` (CPU) performs the model forward pass (TPU) and produces the output raw (padded) `audio: Tensor` waveform (CPU). Thus, the benchmarking numbers take data-movement in account as its often the bottleneck for latency-critical applications.

In [52]:
def infer_e2e(text: str, parsed: torch.Tensor | None = None) -> tuple[torch.Tensor, int]:
    '''
    This function can optionally consume a directly parsed tensor
    for debugging purposes.
    '''
    if parsed is None:
        assert type(text) is str, f'Invalid input type. Got: {type(text)} expected string'
        parsed = parse_text(text)

    audio, real_len = fp_prediction(parsed.to(device))
    
    return audio, real_len

## Warmup

We do a **compilation warmup** here to reduce the chances of compilation cache misses. Re-compilations take $> 800-1200 \text{ms}$ thus we want to avoid them as much as possible. 
Note that there still may be some recompilations (which would be obvious from the latency) but with a good enough warmup plus continued usage, those issues should be alleviated as a good enough cache is constructed.

Cache resides in the directly `./compilation_cache` and should be cleared periodically.

In [ ]:
from tqdm import tqdm
import string
import random
import warnings

def generate_random_string(length):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

def warmup_parse_text():
    for length in tqdm(range(128)):
        text = generate_random_string(length)
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                out = parse_text(text)
        except Exception as e:
            continue

warmup_parse_text()

In [13]:
_parsed = torch.randint(0, 50_000, (1, 128)).long().to(device)
infer_e2e(text = None, parsed = _parsed)

(tensor([[-0.0024, -0.0023, -0.0021,  ...,  0.0862,  0.0216, -0.0041]],
        dtype=torch.float16),
 tensor(461., device='xla:0'))

## End-to-end inference

This is the `text` we wish to convert to audio:

In [74]:
text = "The employee wanted to request his employers for a raise, but was unable to do so because he feared his immediate expulsion."

In [75]:
with benchmark_xla('End-to-end forward pass'):
    raw_audio, real_len = infer_e2e(text)

End-to-end forward pass - Time taken: 70.169 ms (0.07 seconds)


One aspect we wish to draw attention upon is the parsing operation of the text itself (performed by `parse_text`) is often the bottleneck, taking up majority of the time-taken ($50-60\%$). The actual forward pass timing is thus multiple factors less than the timing provided above. With further optimizations (lowering the parsing operation into more performance system languages like Rust or C++) we can reduce the latency even more if required.

In [76]:
with benchmark_xla('Text Parsing'):
    _ = parse_text(text)

Text Parsing - Time taken: 43.942 ms (0.044 seconds)


Here, we provide a utility to convert this raw audio waveform to an Ipython-embedded widget for easy playback on the browser

In [77]:
from IPython.display import Audio
audio = raw_audio.cpu().detach().numpy()[:, :real_len.long().item() * 256]

Audio(audio, rate=22050)